# Build pipeline with registered components

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define `CommandComponent` using YAML, `command_component` decorator
- Create components into workspace
- Create `Pipeline` using registered components.

**Motivations** - This notebook explains different method to create components via SDK then use these components to build pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [1]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

Class SystemCreatedStorageAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SystemCreatedAcrAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [3]:
# Get a handle to workspace
#ml_client = MLClient.from_config(credential=credential)

subscription_id = "<subscription_id>"
resource_group = "<resource_group>"
workspace = "<AML workspace>"

ml_client = MLClient(credential, subscription_id, resource_group, workspace)
# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
#print(ml_client.compute.get(cluster_name))

Class RegistryOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


# 2. Define and create components into workspace
## 2.1 Load components definition from YAML and Register components into workspace

#### 

In [5]:
parent_dir = "."

#### Create Data Extraction Components

In [ ]:
data_extraction = load_component(source=parent_dir + "/data_extraction.yml")

try:
    data_extraction = ml_client.components.get(name="data_extraction", version="1.5")
except:
    data_extraction = ml_client.components.create_or_update(data_extraction)


print(data_extraction)

#### Create Bing Data Enrichment Components

In [ ]:
data_enrichment = load_component(source=parent_dir + "/data_enrichment.yml")

try:
    data_enrichment = ml_client.components.get(name="data_enrichment", version="1.7")
except:
    data_enrichment = ml_client.components.create_or_update(data_enrichment)


print(data_enrichment)

#### Create OpenAI Data Enrichment Components

In [ ]:
openai_enrichment = load_component(source=parent_dir + "/data_enrichment_openAI.yml")

try:
    openai_enrichment = ml_client.components.get(name="openai_enrichment", version="1.0.5")
except:
    openai_enrichment = ml_client.components.create_or_update(openai_enrichment)


print(openai_enrichment)

#### Create Data Clustering Components

In [ ]:
data_pair_clustering = load_component(source=parent_dir + "/clustering_pipeline.yml")

try:
    data_pair_clustering = ml_client.components.get(name="data_pair_clustering", version="1.0.2")
except:
    data_pair_clustering = ml_client.components.create_or_update(data_pair_clustering)


print(data_pair_clustering)

#### Create Data Visulization Components

In [ ]:
data_visualization = load_component(source=parent_dir + "/data_visualization.yml")

try:
    data_visualization = ml_client.components.get(name="data_visualization", version="1.0.3")
except:
    data_visualization = ml_client.components.create_or_update(data_visualization)


print(data_visualization)


##### Check if the enviroment is available and ge the enviroment id 

In [25]:
envs = ml_client.environments.get(name="docker-image-for-pair-matching", version="2")
print(envs)

Environment({'is_anonymous': False, 'auto_increment_version': False, 'name': 'docker-image-for-pair-matching', 'description': 'Environment created from a Docker image plus pair matching packages.', 'tags': {}, 'properties': {}, 'id': '/subscriptions/aa18b01c-698a-4766-8181-9121aa576dc4/resourceGroups/rs1/providers/Microsoft.MachineLearningServices/workspaces/ymao-ws1/environments/docker-image-for-pair-matching/versions/2', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/yuexinmao1/code/Users/yuexinmao/collectioin-ym/pipeline_comfort_poc', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f756cd943d0>, 'serialize': <msrest.serialization.Serializer object at 0x7f7594542040>, 'version': '2', 'latest_version': None, 'conda_file': {'channels': ['conda-forge'], 'dependencies': ['python=3.8', 'pip=21.2.4', {'pip': ['jellyfish==0.9.0', 'joblib==1.2.0', 'numpy==1.23.4', 'pandas==1.5.1', 'python-dateutil==2.8.2', 'pytz==20

# 3. pipeline job
## 3.1 Build pipeline

##### data extraction, form recognizer pipeline

### Create the Pipeline

In [59]:
from azure.ai.ml.constants import AssetTypes

# Construct pipeline
@pipeline()
def pipeline_data_extraction(
    data_extraction_start_index,
    data_extraction_end_index,
    block_attributes,
    cluster_attributes,
    similarity_threshold,
    openai_engine


 ):
    """Comfort PoC Vendor Invoice Analysis."""
    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    Form_recognizer_Process = data_extraction(
        start_index = data_extraction_start_index,
        end_index = data_extraction_end_index
    )

    Bing_Search_Process = data_enrichment(
        file_path_in = Form_recognizer_Process.outputs.output
    )

    OpenAI_Data_Enrichment_Process = openai_enrichment(
        file_path_in = Bing_Search_Process.outputs.file_path_out,
        gpt_engine = openai_engine
    )

    Vendor_Product_Clustering_Process = data_pair_clustering(
        input = OpenAI_Data_Enrichment_Process.outputs.file_path_out,
        block_attributes = block_attributes,
        cluster_attributes = cluster_attributes,
        similarity_threshold=similarity_threshold
    )

    Data_Summarization_for_PowerBI = data_visualization(
        input = Vendor_Product_Clustering_Process.outputs.output

    )

    # Return: pipeline outputs
    return {
        "outputs": Vendor_Product_Clustering_Process.outputs.output
    }
pipeline_job = pipeline_data_extraction(
    data_extraction_start_index = 0,
    data_extraction_end_index = 2777,
    block_attributes = "file_type",
    cluster_attributes = "file_type Description",
    similarity_threshold= 0.9,
    openai_engine= 'summ-davinci002'
    #file_path_out = Input(type="uri_folder", path=parent_dir + "/data/output/"),
)

# set pipeline level compute
pipeline_job.settings.default_compute = "<Add Compute Name>"

# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="InformationExtraction_OpenAI"
)
pipeline_job

Experiment,Name,Type,Status,Details Page
InformationExtraction_OpenAI,cyan_mango_6jppdy8th2,pipeline,Preparing,Link to Azure Machine Learning studio


# Next Steps
You can see further examples of running a pipeline job [here](../)